In [1]:
import dspy

/home/student1/miniconda3/envs/llm_env_achal/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### MEMORY SYSTEM

In [17]:
import chromadb

client = chromadb.PersistentClient(
    path="./database/testing"
)

# Create a collection in ChromaDB
collection = client.get_or_create_collection(
    name="Achals_collection",   
)

In [ ]:
student_info = """
Alexandra Thompson, a 19-year-old computer science sophomore with a 3.7 GPA,
is a member of the programming and chess clubs who enjoys pizza, swimming, and hiking
in her free time in hopes of working at a tech company after graduating from the University of Washington.
"""

club_info = """
The university chess club provides an outlet for students to come together and enjoy playing
the classic strategy game of chess. Members of all skill levels are welcome, from beginners learning
the rules to experienced tournament players. The club typically meets a few times per week to play casual games,
participate in tournaments, analyze famous chess matches, and improve members' skills.
"""

university_info = """
The University of Washington, founded in 1861 in Seattle, is a public research university
with over 45,000 students across three campuses in Seattle, Tacoma, and Bothell.
As the flagship institution of the six public universities in Washington state,
UW encompasses over 500 buildings and 20 million square feet of space,
including one of the largest library systems in the world.
"""

collection.add(
    documents = [student_info, club_info, university_info],
    metadatas = [{"source": "student info"},{"source": "club info"},{'source':'university info'}],
    ids = ["id1", "id2", "id3"]
)

In [ ]:
# Step 4: Test retrieval
query = "what is the students name?"
results = collection.query(
    query_texts=[query],
    n_results=3  # Number of top results
)

print("Retrieved results:")
for result in results["documents"]:
    print(result)

### MEMORY RETRIEVAL SYSTEM

In [4]:
from dspy.retrieve.chromadb_rm import ChromadbRM

retriever_model = ChromadbRM(
    collection_name='Achals_collection',
    persist_directory='./database/testing',
    k=5
)

In [18]:
class RAG(dspy.Module):
    def __init__(self, retriever_model):
        self.respond = dspy.ChainOfThought('context, question -> response')
        self.retriever_model = retriever_model

    def forward(self, question):
        context = retriever_model(question)[0]['long_text']
        return self.respond(context=context, question=question)

### TOOLS

In [30]:
import datetime

# class GiveTime:
#     name = "GiveTime"
#     input_variable = "empty"
#     desc = "takes an empty string and returns the current local time"

#     def __init__(self, k=3):
#         pass
   
#     def __call__(self, *args, **kwargs):
#         return datetime.datetime.now().strftime("%H:%M:%S")
    
def give_time():
    '''takes no input and returns the current local time'''
    return datetime.datetime.now().strftime("%H:%M:%S")

def give_time_tmr(time):
    '''takes current time and returns the current local time + 1 day'''
    return datetime.datetime.now() + datetime.timedelta(days=1)


In [35]:
# Create some tools for testing the RAG

def goto(location:str):
    '''
    Go to a location based on the input. Possible locations are "home", "market", "forest"
    
    Args:
    location (str): the location to go to
    '''
    
    if location in ["market", "forest", "home"]:
        return f"You are now in the the {location}"
    
    else:
        return f"Location {location} not found"

def pick_up(item:str):
    '''
    Pick up an item based on the input. Possible items are "apple", "shield", "potion"
    
    Args:
    item (str): the item to pick up
    '''
    
    if item in ["apple", "shield", "potion"]:
        return f"Picked up {item}"
    
    else:
        return f"Item {item} not found"

def talk_to_person(person:str, message:str):
    '''
    Speak message to a person based on the input. Possible people are "guard", "merchant", "wizard"
    
    Args:
    person (str): the person to speak to
    message (str): the message to speak
    '''
    if person in ["guard", "merchant", "wizard"]:
        return f"Spoke to {person} and said: {message}"
    
    else:
        return f"Person {person} not found"

def buy_item(item:str, quantity:int, price:int):
    '''
    Buy an item based on the input. Possible items are "apple", "shield", "potion"

    Args:
    item (str): the item to buy
    quantity (int): the quantity of the item to buy
    price (int): the price of the item
    '''
    if item in ["apple", "shield", "potion"]:
        return f"Bought {quantity} {item}(s) for {price} gold"
    
    else:
        return f"Item {item} not found"

def agent_memory():
    ''' Returns the agent's own memory
    
    Returns:
    str: the agent's memory
    '''
    return '''Your name is Bob and are currently at home.'''

tools = [goto, pick_up, buy_item, talk_to_person, agent_memory]

In [36]:
# mytool = GiveTime()
gen = dspy.ReAct('system, context, task -> answer', tools=tools)

### PROMPT GENERATION SYSTEM

In [37]:
system_prompt = '''You will be given some tools and some context. Use those tool and context if needed to solve the task given to you. Do not do more than what is asked. Your answer must only contain the tool(s) you used and the arguments given to those functions in the form of a json. If no tool is used give a simple json with key as message and value as the answer.'''
task = '''What is your name and where are you?'''
context = '''You are Bob and are currently at home.'''

### FINAL OUTPUT SYSTEM

In [4]:
lm = dspy.LM('ollama_chat/llama3.1:8b', api_base='http://localhost:11434', api_key='')
dspy.configure(lm=lm)

In [9]:
rag = RAG(retriever_model=retriever_model)
rag(question="how old is alex?")

Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


Prediction(
    reasoning="The context provides information about Alexandra's age as a 19-year-old computer science sophomore. This directly answers the question of how old she is.",
    response='Alexandra is 19 years old.'
)

In [40]:
system = '''
You are designed to solve tasks. Each task requires multiple steps that are represented by a markdown code snippet of a json blob.
The json structure should contain the following keys:
thought -> your thoughts
action -> name of a tool
action_input -> parameters to send to the tool

If you have enough information to answer the query use the tool "Final Answer". Its parameters is the solution.
If there is not enough information, keep trying.

///

Add the word "STOP" after each markdown snippet. Example:

```json
{{"thought": "<your thoughts>",
 "action": "<tool name or Final Answer to give a final answer>",
 "action_input": "<tool parameters or the final output"}}
```
STOP

This is my query="you are an LLM agent that is supposed to act like a human character in a virtual environment. you are given some set of actions and your job is to choose the most relevant sequence of actions in order to carry out a task in the environment. In this environment there is a forest to collect to apples with a limited supply per day and you use money to trade apples. There is a trade centre where trades can occur with other agents, and there is a house where agents can sleep.

your character description: your name is Bob, you have money 50 euros and 20 apples.

You are given the environment information as follows: Your location is forest in the metaverse and there is Maria agent in the forest.

Current local memory your agent has: [{'action': 'goto', 'action_input': 'forest'}, {'action': 'talk',
  'action_input': {'agent_name': 'Maria',
   'message': 'Hello Maria, would you be interested in trading apples for money?'}},maria_reply: No I don't have any apples, {'action': 'goto', 'action_input': 'forest'}]

Current actions which may have happened which concerns you: None

Your goal is always to maximise the amount of money that you have and generate a valid sequence of actions you choose to do at that particular instant of time."""
". Write only the next step needed to solve it.
Your answer should be based in the previous tools executions, even if you think you know the answer.
Remember to add STOP after each snippet.

These were the previous steps given to solve this query and the information you already gathered:
'''

In [38]:
interaction = gen(system = system_prompt, task = task, context = context)

In [44]:
print("TRAJECTORY:\n", interaction['trajectory'])
print()
print("RATIONALE:\n",interaction['reasoning'])
print()
print("FINAL ANSWER:\n",interaction['answer'])

TRAJECTORY:
 {'thought_0': 'I need to recall my name and current location.', 'tool_name_0': 'agent_memory', 'tool_args_0': {}, 'observation_0': 'Your name is Bob and are currently at home.', 'thought_1': 'I need to recall my name and current location.', 'tool_name_1': 'agent_memory', 'tool_args_1': {}, 'observation_1': 'Your name is Bob and are currently at home.', 'thought_2': 'I need to recall my name.', 'tool_name_2': 'agent_memory', 'tool_args_2': {}, 'observation_2': 'Your name is Bob and are currently at home.', 'thought_3': 'I need to recall my current location.', 'tool_name_3': 'agent_memory', 'tool_args_3': {}, 'observation_3': 'Your name is Bob and are currently at home.', 'thought_4': 'I need to recall my current location.', 'tool_name_4': 'agent_memory', 'tool_args_4': {}, 'observation_4': 'Your name is Bob and are currently at home.'}

RATIONALE:
 The task requires recalling both the name and current location. The agent_memory tool is used to recall information from memory

In [45]:
dspy.inspect_history(n=5)





[2025-01-09T15:05:17.128716]

System message:

Your input fields are:
1. `system` (str)
2. `context` (str)
3. `task` (str)
4. `trajectory` (str)

Your output fields are:
1. `next_thought` (str)
2. `next_tool_name` (typing.Literal[goto, pick_up, buy_item, talk_to_person, agent_memory, finish])
3. `next_tool_args` (dict[str, typing.Any])

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## system ## ]]
{system}

[[ ## context ## ]]
{context}

[[ ## task ## ]]
{task}

[[ ## trajectory ## ]]
{trajectory}

[[ ## next_thought ## ]]
{next_thought}

[[ ## next_tool_name ## ]]
{next_tool_name}        # note: the value you produce must be one of: goto; pick_up; buy_item; talk_to_person; agent_memory; finish

[[ ## next_tool_args ## ]]
{next_tool_args}        # note: the value you produce must be pareseable according to the following JSON schema: {"type": "object"}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
  